#### Load dataset

In [12]:
import pandas as pd
import numpy as np
from lib.data import load_dataset

plain = False # Quero só a imagem serializada?
if plain:
    path = 'dts/dts_plain/'
else:
    path = 'dts/dts_all/'
x_train, y_train, x_valid, y_valid = load_dataset(path + 'train.csv', rs=2)

#### Definição da pipeline
- Definição do seletor de features
- Definição do random forest classifier

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
rs = 2
# forest = RandomForestClassifier(random_state=rs, n_estimators=20)
forest = Pipeline([
    # ('standardscaler', StandardScaler()),
    ('feature_selection', SelectFromModel(estimator=LogisticRegression(random_state=rs, max_iter=2000)).fit(x_train, y_train)),
    ('classification', RandomForestClassifier(random_state=rs, n_estimators=20))
])

KeyboardInterrupt: 

#### Treino do algoritmo

In [14]:
forest.fit(x_train, y_train)
y_pred = forest.predict(x_valid)

d:\ProgramData\Anaconda3\envs\ACA\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Display das metrics (Accuracy, RoC, precision, recall)

In [15]:
from lib.metrics import metrics
metrics(y_pred, y_valid)

Metrics: 
 Acc: 0.982, ROC: 0.982
 PRE: 0.988, REC: 0.977


##### Load dataset de teste

In [16]:
df2 = pd.read_csv(path + "/test.csv")

x_test = df2.iloc[:, :-1]
y_pred = forest.predict(x_test)

#### Exporta resultado para um arquivo .csv

In [18]:
from lib.data import export_results
result = pd.DataFrame(data={
        "Id": range(y_pred.shape[0]),
        "Category": y_pred.astype(int)
    }, index=None)

result.to_csv("result.csv", index=None)